In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
from sklearn.preprocessing import MinMaxScaler
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import load_model

Using TensorFlow backend.


In [2]:
directory_in_str = os.getcwd()
directory_in_str += '\data_by_id'

In [3]:
def LSTM_train(filename):
    LSTM_model()
    filename = 'data_by_id\\'+filename
    df = pd.read_csv(filename, index_col="time",parse_dates=True)
    df = df[4:]
    column = df['adjusted_open'].count()
    if (column > 200):
        actual_train(df, column)
        # Fitting the RNN to the Training set

In [4]:
def actual_train(df, column):
        training_set = df.drop(['id', 'adjusted_volume'], axis=1)
        # Feature Scaling
        sc = MinMaxScaler(feature_range = (0, 1))
        training_set_scaled = sc.fit_transform(training_set)
        training_set_scaled.shape
        # Creating a data structure with 60 timesteps and 1 output
        X_train = []
        y_train = []
        for i in range(60, column):
            X_train.append(training_set_scaled[i-60:i, :])
            y_train.append(training_set_scaled[i, :])
        X_train, y_train = np.array(X_train), np.array(y_train)
        # Reshaping
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 5))
        regressor = load_model('model.h5')
        regressor.fit(X_train, y_train, epochs = 10, batch_size = 32)
        regressor.save('model.h5')

In [5]:
def LSTM_model():
    # Initialising the RNN
    regressor = Sequential()

    # Adding the first LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (60, 5)))
    regressor.add(Dropout(0.2))

    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    # Adding a third LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    # Adding a fourth LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))

    # Adding the output layer
    regressor.add(Dense(units = 5))
    
    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    regressor.save('model.h5')

In [6]:

# Part 3 - Making the predictions and visualising the results

# Getting the real stock price of 2017
#dataset_test = pd.read_csv('Google_Stock_Price_Test.csv',index_col="Date",parse_dates=True)

In [7]:
directory = os.fsencode(directory_in_str)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"):
        print(filename)
        LSTM_train(filename)

0.csv
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4493/4493 [==============================] - 13s 3ms/step - loss: 0.0052
Epoch 2/10
4493/4493 [==============================] - 10s 2ms/step - loss: 0.0019
Epoch 3/10
4493/4493 [==============================] - 10s 2ms/step - loss: 0.0017
Epoch 4/10
4493/4493 [==============================] - 10s 2ms/step - loss: 0.0014
Epoch 5/10
4493/4493 [==============================] - 10s 2ms/step - loss: 0.0012
Epoch 6/10
4493/4493 [==============================] - 10s 2ms/step - loss: 9.7639e-04
Epoch 7/10
4493/4493 [==============================] - 10s 2ms/step - loss: 8.5178e-04
Epoch 8/10
4493/4493 [==============================] - 10s 2ms/step - loss: 7.7316e-04
Epoch 9/10
4493/4493 [==============================] - 10s 2ms/step - loss

6201/6201 [==============================] - 31s 5ms/step - loss: 9.7098e-04
Epoch 6/10
6201/6201 [==============================] - 31s 5ms/step - loss: 8.0805e-04
Epoch 7/10
6201/6201 [==============================] - 31s 5ms/step - loss: 7.5462e-04
Epoch 8/10
6201/6201 [==============================] - 31s 5ms/step - loss: 6.8070e-04
Epoch 9/10
6201/6201 [==============================] - 31s 5ms/step - loss: 6.0679e-04
Epoch 10/10
6201/6201 [==============================] - 31s 5ms/step - loss: 6.1700e-04
10005.csv
Epoch 1/10
6992/6992 [==============================] - 48s 7ms/step - loss: 0.0115
Epoch 2/10
6992/6992 [==============================] - 35s 5ms/step - loss: 0.0037
Epoch 3/10
6992/6992 [==============================] - 35s 5ms/step - loss: 0.0027
Epoch 4/10
6992/6992 [==============================] - 36s 5ms/step - loss: 0.0022
Epoch 5/10
6992/6992 [==============================] - 35s 5ms/step - loss: 0.0018
Epoch 6/10
6992/6992 [==============================

KeyboardInterrupt: 

In [ ]:
X_test = []
y_test = []
for i in range(column-100, column):
    X_test.append(training_set_scaled[i-60:i, :])
    y_test.append(training_set_scaled[i, :])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 6))
y_predicted = regressor.predict(X_test)
#predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
y_predicted = pd.DataFrame(y_predicted)
y_predicted.info()

In [ ]:
# Visualising the results
plt.plot(training_set_scaled[column-100:, 0], color = 'red', label = 'Real Google Stock Price')
plt.plot(y_predicted[0], color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

In [ ]:
y_real = sc.inverse_transform(y_predicted)
# y_real.shape
training_set.tail(100)['adjusted_open'].values

plt.close()
plt.plot(training_set.tail(100)['adjusted_open'].values, color = 'red', label = 'Real Google Stock Price')
plt.plot(y_real[:, 0], color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

In [ ]:
# Try add/remove layers
# Change dimensionality of layers
# Split training/test/validation
# Test on specific stocks 
# Combine all stocks into single dataset (60day windows for all stocks)
# Change output dimensionality (predict only open)

In [ ]:
def change_category(now, week_from_now):
    ratio = float(week_from_now)/now
    percentile = (ratio - 1) * 100
    if abs(percentile) >= 5:
        return np.sign(percentile) * 3
    elif abs(percentile) >= 3:
        return np.sign(percentile) * 2
    elif abs(percentile) >= 2:
        return np.sign(percentile)
    else:
        return 0